<a href="https://colab.research.google.com/github/Azulita/Garantia-Extendida/blob/master/02_scraping_glassdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Usaremos este notebook para hacer web scraping para encontrar nuestro trabajo ideal. La página de Glassdoor ofrece un catálogo de vacantes en diferentes empresas y nos gustaría poder descargar toda la información para poder analizarla después.

Escribamos un código en Python para descargar toda la información que podamos y que lo podamos ejecutar en distintas ocasiones para actualizar nuestros datos.

Empecemos por instalar algunas librerías en caso de no tenerlas. Estas librerías nos servirán para replicar el comportamiento de un navegador (necesario en algunas páginas para que podamos extraer el código HTML) y para manipular el código HTML de una página y extraer información valiosa.

In [ ]:
# !pip install beautifulsoup4
# !pip install selenium

In [ ]:
import requests
import time

from bs4 import BeautifulSoup
from selenium import webdriver

Una vez instaladas las librerías, hagamos una consulta a una página de Glassdoor. En este caso, la URL se consiguió entrando a la página misma de Glassdoor, realizando una búsqueda y copiando la URL.

In [ ]:
# Genera un "driver", replicando a Google Chrome
browser = webdriver.Chrome()

# Realiza una consulta GET para obtener el código HTML
browser.get("https://www.glassdoor.com.mx/Empleo/data-analyst-empleos-SRCH_KO0,12.htm")

# Obtiene el código fuente de la consulta GET
html = browser.page_source
time.sleep(2)

# Cierra el navegador
browser.close()

# Análisis gramatical del código HTML
soup = BeautifulSoup(html)

# Web scraping

Ahora, pasamos a extraer información valiosa del código HTML de la página. Extraeremos la información más valiosa, la iremos guardando primero en listas y después guardaremos la información en un DataFrame de Pandas al final.

In [ ]:
import pandas as pd

company_names = []
company_ratings = []
job_titles = []
job_links = []
job_ids = []
locations = []
salary_ranges = []
is_fast_candidacies = []
listing_ages = []

job_cards = soup.find_all("li", attrs={"class": "JobsList_jobListItem__JBBUV"})

for job in job_cards:

    company_info = job.find("div", attrs={"class": "EmployerProfile_employerInfo__GaPbq"})

    try:
        company_rating_info = company_info.find("span", attrs={"class": "EmployerProfile_employerRating__3ADTJ"})
    except:
        continue

    company_name = company_info.find_next(text=True)
    try:
        company_rating = company_rating_info.text.strip()
    except:
        company_rating = None

    job_title_info = job.find("a", attrs={"class": "JobCard_seoLink__WdqHZ"})
    job_title = job_title_info.text
    job_link = job_title_info.get("href")
    job_id = job_title_info.get("id").replace("job-title-", "")

    location = job.find("div", attrs={"class": "JobCard_location__N_iYE"}).text

    # Limpiar: calcular límite inferior y superior
    salary_range_info = job.find("div", attrs={"class": "JobCard_salaryEstimate___m9kY"})
    try:
        salary_range = salary_range_info.text.replace("(Est. del empleador)", "").replace("\xa0", "").strip()
    except:
        salary_range = None

    # Limpiar: cambiar a 0 o 1
    try:
        is_fast_candidacy = job.find("div", attrs={"class": "JobCard_easyApply___eIoB"}).text
    except:
        is_fast_candidacy = None

    # Limpiar: sustituir horas por días, cambiar todo a enteros
    listing_age = job.find("div", attrs={"class": "JobCard_listingAge__KuaxZ"}).text.replace("\xa0d", "")

    company_names.append(company_name)
    company_ratings.append(company_rating)
    job_titles.append(job_title)
    job_links.append(job_link)
    job_ids.append(job_id)
    locations.append(location)
    salary_ranges.append(salary_range)
    is_fast_candidacies.append(is_fast_candidacy)
    listing_ages.append(listing_age)

jobs_data = pd.DataFrame(data={
    "company_name": company_names,
    "company_rating": company_ratings,
    "job_title": job_titles,
    "job_link": job_links,
    "job_id": job_ids,
    "location": locations,
    "salary_range": salary_ranges,
    "is_fast_candidacy": is_fast_candidacies,
    "listing_age": listing_ages,
})

In [ ]:
jobs_data.head()

## Sanity Checks

In [ ]:
jobs_data.shape

In [ ]:
jobs_data["company_rating"].astype("float").mean()

In [ ]:
(jobs_data["is_fast_candidacy"] == "Candidatura rápida").mean()

# Guardar los datos

In [ ]:
from datetime import date

today = str(date.today())[:10]

jobs_data.to_csv(f"{today}_jobs_data.csv", index=False)

# Tarea

Existe un botón de `"Mostrar más empleos"` en la página consultada. Usando Selenium, repliquen el evento de dar click en ese botón para que aparezcan más resultados y vuelvan a extraer la información una vez que hayan logrado esto. Deberán de obtener una tabla con los 30 resultados de antes más resultados nuevos (aproximadamente otros 30 resultados).

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

browser = webdriver.Chrome()

## 1. Obtener el código HTML incial
browser.get("https://www.glassdoor.com.mx/Empleo/data-analyst-empleos-SRCH_KO0,12.htm")

## 2. instrucción para esperar a que cargue el botón que queremos
time.sleep(5)
selector = '//button[@data-test="load-more"]'

wait = WebDriverWait(browser, 10)
button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
time.sleep(2)
button.click()

## 3. extraer el código HTML de la página después de haber dado click
time.sleep(5)
html = browser.page_source
time.sleep(2)

browser.close()

soup = BeautifulSoup(html)

In [ ]:
job_cards = soup.find_all("li", attrs={"class": "JobsList_jobListItem__JBBUV"})
len(job_cards)

# Tiene que salir este resultado: 60 o 61

Ahora, copien y peguen las instrucciones para extraer la información de la página y verifiquen que tienen más de 30 filas en su DataFrame.

In [ ]:
# Copien y peguen el código de la sección "Web Scraping" aquí:
# -------------------------- TAREA --------------------------


In [ ]:
jobs_data.head()